In [1]:
import pandas as pd
import scraper

c:\Users\proso\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
scraper.generate_CIK_TICKER()

In [3]:
recentFilings = pd.read_csv('ticker-SEC.csv')

In [4]:
ticker = 'AAPL'

selectedTicker = recentFilings[recentFilings['ticker'] == ticker]
selectedCik = selectedTicker.cik_str.values[0]

In [5]:

url = f'https://www.sec.gov/edgar/browse/?CIK={selectedCik}'
scr = scraper.APIconnector(url)
#soup = scr.make_soup()
#soup

In [6]:
selectedCik = '0000320193'
reqURL = F'https://data.sec.gov/submissions/CIK{selectedCik}.json'

scr.URL = reqURL
res = scr.get_request()
res

<Response [200]>

In [7]:
JSONresponse = res.json()

In [8]:
JSONresponse.keys()

dict_keys(['cik', 'entityType', 'sic', 'sicDescription', 'insiderTransactionForOwnerExists', 'insiderTransactionForIssuerExists', 'name', 'tickers', 'exchanges', 'ein', 'description', 'website', 'investorWebsite', 'category', 'fiscalYearEnd', 'stateOfIncorporation', 'stateOfIncorporationDescription', 'addresses', 'phone', 'flags', 'formerNames', 'filings'])

In [9]:
JSONresponse['insiderTransactionForIssuerExists']

1

In [10]:
#https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json
recentFilings = pd.DataFrame.from_dict(JSONresponse['filings']['recent'])
"""
https://www.sec.gov/ix?doc=/Archives/edgar/data/0000320193/000114036124024352/ef20028273_8k.htm

https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json

/0001140361|24|024352/
accessionNumber - 0001140361-24-024352

"""
recentFilings.head(2)


,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0001140361-24-024352,2024-05-03,2024-05-03,2024-05-03T16:35:42.000Z,34,8-K,001-36743,24913807,"7.01,9.01",1131483,1,1,ef20028273_8k.htm,8-K
1,0000320193-24-000069,2024-05-03,2024-03-30,2024-05-02T18:04:25.000Z,34,10-Q,001-36743,24910117,,5284139,1,1,aapl-20240330.htm,10-Q


In [11]:
recentFilings['accessionNumberCLEAN'] = recentFilings['accessionNumber'].apply(lambda x: x.replace('-', ''))
recentFilings['fileURL'] = 'https://www.sec.gov/Archives/edgar/data/' + selectedCik + "/" + \
    recentFilings['accessionNumberCLEAN'] + "/"+ recentFilings['primaryDocument']

recentFilings

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription,accessionNumberCLEAN,fileURL
0,0001140361-24-024352,2024-05-03,2024-05-03,2024-05-03T16:35:42.000Z,34,8-K,001-36743,24913807,"7.01,9.01",1131483,1,1,ef20028273_8k.htm,8-K,000114036124024352,https://www.sec.gov/Archives/edgar/data/000032...
1,0000320193-24-000069,2024-05-03,2024-03-30,2024-05-02T18:04:25.000Z,34,10-Q,001-36743,24910117,,5284139,1,1,aapl-20240330.htm,10-Q,000032019324000069,https://www.sec.gov/Archives/edgar/data/000032...
2,0000320193-24-000067,2024-05-02,2024-05-02,2024-05-02T16:30:34.000Z,34,8-K,001-36743,24908945,"2.02,9.01",436360,1,1,aapl-20240502.htm,8-K,000032019324000067,https://www.sec.gov/Archives/edgar/data/000032...
3,0000320193-24-000058,2024-04-17,2024-04-15,2024-04-17T18:30:52.000Z,,4,,,,16177,0,0,xslF345X05/wk-form4_1713393040.xml,FORM 4,000032019324000058,https://www.sec.gov/Archives/edgar/data/000032...
4,0000320193-24-000056,2024-04-15,2024-04-11,2024-04-15T18:31:11.000Z,,4,,,,5376,0,0,xslF345X05/wk-form4_1713220262.xml,FORM 4,000032019324000056,https://www.sec.gov/Archives/edgar/data/000032...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0001181431-14-000951,2014-01-03,2013-12-31,2014-01-03T18:36:12.000Z,,4,,,,5012,0,0,xslF345X03/rrd397763.xml,13.12.31 RICCIO FORM 4 - 10B5-1 SALE,000118143114000951,https://www.sec.gov/Archives/edgar/data/000032...
996,0001193125-13-486406,2013-12-27,2014-02-28,2013-12-27T17:01:57.000Z,34,PRE 14A,000-10030,131301152,,1584302,0,0,d648739dpre14a.htm,PRELIMINARY PROXY STATEMENT,000119312513486406,https://www.sec.gov/Archives/edgar/data/000032...
997,0001181431-13-064578,2013-12-26,2013-12-23,2013-12-26T18:32:35.000Z,,4,,,,8519,0,0,xslF345X03/rrd397746.xml,13.12.23 FEDERIGHI FORM 4 - RSU VESTING,000118143113064578,https://www.sec.gov/Archives/edgar/data/000032...
998,0001181431-13-064577,2013-12-26,2013-12-23,2013-12-26T18:30:47.000Z,,4,,,,8519,0,0,xslF345X03/rrd397758.xml,13.12.23 RICCIO FORM 4 - RSU VESTING,000118143113064577,https://www.sec.gov/Archives/edgar/data/000032...


In [12]:
testURL = recentFilings[recentFilings['form'] == '10-Q']['fileURL'].to_numpy()[1]

scr.URL = testURL
res = scr.make_soup()
res

<?xml version='1.0' encoding='ASCII'?>
<html xml:lang="en-US" xmlns="http://www.w3.org/1999/xhtml" xmlns:aapl="http://www.apple.com/20231230" xmlns:dei="http://xbrl.sec.gov/dei/2023" xmlns:ecd="http://xbrl.sec.gov/ecd/2023" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:ix="http://www.xbrl.org/2013/inlineXBRL" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2020-02-12" xmlns:ixt-sec="http://www.sec.gov/inlineXBRL/transformation/2015-08-31" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:srt="http://fasb.org/srt/2023" xmlns:us-gaap="http://fasb.org/us-gaap/2023" xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><head><meta content="text/html" http-equiv="Content-Type"/>
<title>aapl-20231230</title><script>bazadebezolkohpepadr="1000777775"</script><script defer="" src="https://www.sec.gov/akam/13/3ba6aa31" type="text/javascript"><

In [13]:
res.prettify()

'<?xml version=\'1.0\' encoding=\'ASCII\'?>\n<html xml:lang="en-US" xmlns="http://www.w3.org/1999/xhtml" xmlns:aapl="http://www.apple.com/20231230" xmlns:dei="http://xbrl.sec.gov/dei/2023" xmlns:ecd="http://xbrl.sec.gov/ecd/2023" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:ix="http://www.xbrl.org/2013/inlineXBRL" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2020-02-12" xmlns:ixt-sec="http://www.sec.gov/inlineXBRL/transformation/2015-08-31" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:srt="http://fasb.org/srt/2023" xmlns:us-gaap="http://fasb.org/us-gaap/2023" xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n <head>\n  <meta content="text/html" http-equiv="Content-Type"/>\n  <title>\n   aapl-20231230\n  </title>\n  <script>\n   bazadebezolkohpepadr="1000777775"\n  </script>\n  <script defer="" src="https://www.sec.gov/

In [14]:
scr.response.text

'<?xml version=\'1.0\' encoding=\'ASCII\'?>\n<html xmlns:ecd="http://xbrl.sec.gov/ecd/2023" xmlns:ix="http://www.xbrl.org/2013/inlineXBRL" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:aapl="http://www.apple.com/20231230" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2020-02-12" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:us-gaap="http://fasb.org/us-gaap/2023" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xmlns:dei="http://xbrl.sec.gov/dei/2023" xmlns:ixt-sec="http://www.sec.gov/inlineXBRL/transformation/2015-08-31" xmlns="http://www.w3.org/1999/xhtml" xmlns:srt="http://fasb.org/srt/2023" xml:lang="en-US"><head><meta http-equiv="Content-Type" content="text/html"/>\n\n\n<title>aapl-20231230</title><script >bazadebezolkohpepadr="1000777775"</script><script type="text/javascript" src="https://www.sec.gov/akam/13/3ba6aa31

In [27]:
res.find_all('table')[7].text

'Title of each classTrading symbol(s)Name of each exchange on which registeredCommon Stock, $0.00001 par value per shareAAPLThe Nasdaq Stock Market LLC0.000% Notes due 2025—The Nasdaq Stock Market LLC0.875% Notes due 2025—The Nasdaq Stock Market LLC1.625% Notes due 2026—The Nasdaq Stock Market LLC2.000% Notes due 2027—The Nasdaq Stock Market LLC1.375% Notes due 2029—The Nasdaq Stock Market LLC3.050% Notes due 2029—The Nasdaq Stock Market LLC0.500% Notes due 2031—The Nasdaq Stock Market LLC3.600% Notes due 2042—The Nasdaq Stock Market LLC'

In [15]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession


import nest_asyncio

nest_asyncio.apply()

s = HTMLSession()

r = s.get(testURL)
r.text

'<?xml version=\'1.0\' encoding=\'ASCII\'?>\n<html xmlns:ecd="http://xbrl.sec.gov/ecd/2023" xmlns:ix="http://www.xbrl.org/2013/inlineXBRL" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:aapl="http://www.apple.com/20231230" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2020-02-12" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:us-gaap="http://fasb.org/us-gaap/2023" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xmlns:dei="http://xbrl.sec.gov/dei/2023" xmlns:ixt-sec="http://www.sec.gov/inlineXBRL/transformation/2015-08-31" xmlns="http://www.w3.org/1999/xhtml" xmlns:srt="http://fasb.org/srt/2023" xml:lang="en-US"><head><meta http-equiv="Content-Type" content="text/html"/>\n\n\n<title>aapl-20231230</title></head><body><div style="display:none"><ix:header><ix:hidden><ix:nonNumeric contextRef="c-1" name="dei:AmendmentFlag" id="f

In [16]:
testURL

'https://www.sec.gov/Archives/edgar/data/0000320193/000032019324000006/aapl-20231230.htm'